---

# **Moore MKW - Goods Transfer Anomaly Detection**
### **A pilot project using Baks data**

### Moni IT & Consultancy

### Gert Wijnalda, M.M.J.D.T. Voeten

---

---

# **1.0 - Project Goal**

---





- Check whether transport data contains anomalies
- Check the transport company as well as the logistics services provider

---

# **2.0 - Dataset Description**

---

---

# **3.0 - Methodology**

---

---

# **4.0 - Imports & Preliminaries**

---

In [706]:
%pip install -r "requirements.txt"

Note: you may need to restart the kernel to use updated packages.


In [707]:
# General
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from haversine import haversine, Unit
import re

# Visualisations
import matplotlib.pyplot as pltå
import seaborn as sns
import plotly.express as px
import matplotlib.ticker as mtick

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import ttest_rel, wilcoxon, pearsonr
from sklearn.ensemble import IsolationForest
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack

# Warnings
import warnings
warnings.filterwarnings('ignore')

In [708]:
magic_number = sum([ord(x) for x in "Make Moni Flow"])

SEED = magic_number
TEST_SIZE = 0.2
RANDOM_STATE = magic_number
print(magic_number)

1257


In [709]:
raw_data = "./src/Baks Logistiek - 2025.xlsx"
dataset = pd.read_excel(raw_data)
dataset.head()

,Dossiernr,Volgnr,Referentie,Zendingsoort,Klaar voor factuur,Totaalbedrag,Extra kosten bedrag,Brandstoftoesl. extra kosten,Brandstoftoeslag bedrag,Bedrag excl.,...,Duur laden,Calculatie gewicht,Eindtijd lossen,Einddatum laden,Eindtijd laden,Einddatum lossen,Startdatum lossen,Starttijd lossen,Startdatum laden,Starttijd laden
0,551801,1,NaN,2 - Factuur akkoord,Ja,997.19,312.40,NaN,73.37,611.42,...,30.0,33.10,1900-01-01 12:45:01,2025-01-01,1900-01-01 05:45:01,2025-01-01,2025-01-01,1900-01-01 09:11:06,2025-01-01,1900-01-01 05:15:23
1,552062,1,NaN,2 - Factuur akkoord,Ja,1027.70,275.00,NaN,80.65,672.05,...,53.0,NaN,1900-01-01 13:20:56,2025-01-01,1900-01-01 09:46:02,2025-01-01,2025-01-01,1900-01-01 11:06:19,2025-01-01,1900-01-01 08:53:07
2,552063,1,NaN,2 - Factuur akkoord,Ja,1020.98,275.00,NaN,73.93,672.05,...,64.0,NaN,1900-01-01 11:54:21,2025-01-01,1900-01-01 08:03:03,2025-01-01,2025-01-01,1900-01-01 09:28:01,2025-01-01,1900-01-01 06:59:18
3,552076,1,NaN,2 - Factuur akkoord,Ja,2033.90,NaN,NaN,184.90,1849.00,...,107.0,25.52,1900-01-01 20:13:33,2024-12-31,1900-01-01 08:54:39,2024-12-31,2024-12-31,1900-01-01 19:11:03,2024-12-31,1900-01-01 07:07:43
4,552090,1,6100057971,2 - Factuur akkoord,Ja,863.01,226.87,NaN,63.04,573.10,...,177.0,34.86,1900-01-01 14:03:57,2025-01-01,1900-01-01 11:31:20,2025-01-01,2025-01-01,1900-01-01 12:53:02,2025-01-01,1900-01-01 08:34:36


---

# **5.0 - Dataset Cleaning**

---

In [710]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38551 entries, 0 to 38550
Columns: 136 entries, Dossiernr to Starttijd laden
dtypes: datetime64[ns](20), float64(52), int64(7), object(57)
memory usage: 40.0+ MB


In [711]:
dataset.describe()

,Dossiernr,Volgnr,Totaalbedrag,Extra kosten bedrag,Brandstoftoesl. extra kosten,Brandstoftoeslag bedrag,Bedrag excl.,Eenheidsprijs,Vaste prijs,% Droge stof,...,Duur laden,Calculatie gewicht,Eindtijd lossen,Einddatum laden,Eindtijd laden,Einddatum lossen,Startdatum lossen,Starttijd lossen,Startdatum laden,Starttijd laden
count,38551.000000,38551.000000,36823.000000,19090.000000,3588.000000,28134.000000,36343.000000,16291.000000,20080.000000,8893.000000,...,34527.000000,35596.000000,34908,34695,34692,34908,34872,34871,34584,34577
mean,573409.614770,1.146352,719.992897,157.072635,20.448768,65.069750,594.319369,18.093250,648.203013,16.701792,...,80.068121,28.026965,1900-01-01 13:22:58.398246656,2025-04-30 05:55:59.066148096,1900-01-01 12:51:26.127262720,2025-04-30 15:27:02.413200640,2025-04-30 16:02:23.702684160,1900-01-01 12:15:51.479165952,2025-04-30 11:20:16.655100672,1900-01-01 11:38:03.864852224
min,174567.000000,1.000000,-2839.340000,-443.800000,0.000000,-345.050000,-2156.570000,-85.240000,-2141.230000,0.000000,...,0.000000,-35.480000,1900-01-01 00:00:00,2010-01-01 00:00:00,1900-01-01 00:00:00,2010-01-02 00:00:00,2010-01-02 00:00:00,1900-01-01 00:00:00,2010-01-02 00:00:00,1900-01-01 00:00:00
25%,562918.000000,1.000000,388.820000,20.000000,14.880000,33.470000,330.480000,9.850000,340.937500,4.800000,...,51.000000,24.540000,1900-01-01 10:16:10.750000128,2025-03-05 00:00:00,1900-01-01 09:33:56.500000,2025-03-05 00:00:00,2025-03-05 00:00:00,1900-01-01 09:00:21.500000,2025-03-05 00:00:00,1900-01-01 08:18:32
50%,573481.000000,1.000000,577.610000,87.500000,20.250000,57.280000,495.000000,13.450000,575.000000,11.120000,...,68.000000,28.260000,1900-01-01 13:14:05,2025-04-30 00:00:00,1900-01-01 12:36:45,2025-04-30 00:00:00,2025-04-30 00:00:00,1900-01-01 12:05:33,2025-04-30 00:00:00,1900-01-01 11:21:15
75%,583879.500000,1.000000,859.295000,176.240000,25.910000,81.750000,712.790000,20.400000,775.000000,27.420000,...,93.000000,34.034500,1900-01-01 16:16:35.750000128,2025-06-27 00:00:00,1900-01-01 15:55:41.500000,2025-06-27 00:00:00,2025-06-27 00:00:00,1900-01-01 15:12:27.500000,2025-06-27 00:00:00,1900-01-01 14:37:23
max,594713.000000,18.000000,63292.190000,62667.190000,225.270000,741.950000,6745.000000,895.000000,6745.000000,99.000000,...,17259.000000,35000.000000,1900-01-01 23:59:08,2025-08-27 00:00:00,1900-01-01 23:59:26,2025-08-27 00:00:00,2025-08-27 00:00:00,1900-01-01 23:59:46,2025-08-27 00:00:00,1900-01-01 23:57:14
std,12522.573516,0.556865,661.553237,578.117963,10.956177,50.144001,453.613125,16.110707,515.708588,13.503690,...,140.814749,185.550795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [712]:
dataset.memory_usage()

Index                   132
Dossiernr            308408
Volgnr               308408
Referentie           308408
Zendingsoort         308408
                      ...  
Einddatum lossen     308408
Startdatum lossen    308408
Starttijd lossen     308408
Startdatum laden     308408
Starttijd laden      308408
Length: 137, dtype: int64

In [713]:
def feature_summary(df):
    summary = pd.DataFrame({
        'Feature': df.columns,
        'Num_Unique': df.nunique(),
        'Num_Missing': df.isna().sum()
    }).reset_index(drop=True)
    summary = summary.sort_values(by='Num_Missing', ascending=False).reset_index(drop=True)
    return summary

feature_summary(dataset)


,Feature,Num_Unique,Num_Missing
0,Coördinaten laadadres handmatig,0,38551
1,Kritische nippelklant,0,38551
2,Chauffeur laadinfo,0,38551
3,Coördinaten losadres handmatig,0,38551
4,Afwijkend tariefaantal,0,38551
...,...,...,...
131,Legnr,38551,0
132,Departement,2,0
133,Week,35,0
134,Volgnr,18,0


Dataset


In [714]:
data = dataset[['Dossiernr', 'Calculatie gewicht', 'Duur laden', 'Duur lossen', 'Routeberek. resultaat', 'ADR-zending', 'Productomschrijving', 'Financiële datum', 'Artikel merken/nrs.', 'Brandstoftoeslag percentage', 'Gepl. losdatum/tijd', 'Gepl. laaddatum/tijd', 'Aangemaakt op', 'Kosten', 'Losinfo chauffeur', 'Laadinfo chauffeur', 'Losopmerkingen', 'Laadopmerkingen', '# afwijkingen', 'Tariefeenheid', 'Tariefaantal', 'Gewicht', 'Interne opmerkingen', 'Product', 'Afstand', 'Eenheid', 'Aantal', 'Losland', 'Lostijd', 'Losdatum', 'Losplaats', 'Losnaam', 'Laadland', 'Laadtijd', 'Laaddatum', 'Laadplaats', 'Laadnaam', 'Opdrachtgever', 'Losgewicht', 'Laadgewicht', 'Vaste prijs', 'Eenheidsprijs', 'Bedrag excl.', 'Brandstoftoeslag bedrag', 'Brandstoftoesl. extra kosten', 'Extra kosten bedrag', 'Totaalbedrag', 'Breedtegraad laadadres', 'Breedtegraad losadres', 'Lengtegraad laadadres', 'Lengtegraad losadres']]

Referentieset

In [715]:
referentie = dataset[['Oorspr. zendingnr', 'Cargonr', 'EDI referentie', 'Legnr', 'Contactpersoon', 'Factuurnr', 'Laadadres2', 'Losadres2', 'Losadres1', 'Departement', 'Crediteur lossen', 'Trailer lossen', 'Truck lossen', 'Crediteur laden', 'Factuurcode', 'Factuurmethode', 'Gecontroleerd', 'CMR printen', 'Ritnr lossen', 'CMR nr.', 'Laadreferentie', 'Dossiernr']]

In [716]:
data.head(20)

,Dossiernr,Calculatie gewicht,Duur laden,Duur lossen,Routeberek. resultaat,ADR-zending,Productomschrijving,Financiële datum,Artikel merken/nrs.,Brandstoftoeslag percentage,...,Eenheidsprijs,Bedrag excl.,Brandstoftoeslag bedrag,Brandstoftoesl. extra kosten,Extra kosten bedrag,Totaalbedrag,Breedtegraad laadadres,Breedtegraad losadres,Lengtegraad laadadres,Lengtegraad losadres
0,551801,33.10,30.0,214.0,Ok,Ja,Wei proteine concentraat,2025-01-01,Wei proteine concentraat,12.0,...,NaN,611.42,73.37,NaN,312.40,997.19,51.616480,52.117520,5.527540,6.528160
1,552062,NaN,53.0,134.0,Ok,NaN,NaN,2025-01-01,NaN,12.0,...,NaN,672.05,80.65,NaN,275.00,1027.70,52.976005,52.385780,5.454931,5.791300
2,552063,NaN,64.0,146.0,Ok,NaN,NaN,2025-01-01,NaN,11.0,...,NaN,672.05,73.93,NaN,275.00,1020.98,52.976005,52.385780,5.454931,5.791300
3,552076,25.52,107.0,62.0,Ok,Ja,WPC 65,2025-01-01,WPC 65,10.0,...,NaN,1849.00,184.90,NaN,NaN,2033.90,52.976005,56.049790,5.454931,8.668610
4,552090,34.86,177.0,70.0,Ok,NaN,Magere melk,2025-01-01,Magere melk,11.0,...,NaN,573.10,63.04,NaN,226.87,863.01,51.549180,51.831100,5.852160,4.951370
5,552091,34.84,185.0,104.0,Ok,NaN,Magere melk,2025-01-01,Magere melk,11.0,...,NaN,573.10,63.04,NaN,201.22,837.36,51.549180,51.831100,5.852160,4.951370
6,552092,35.68,152.0,115.0,Ok,NaN,Magere melk,2025-01-01,Magere melk,11.0,...,NaN,573.10,63.04,NaN,197.72,833.86,51.549180,51.831100,5.852160,4.951370
7,552106,35.18,69.0,103.0,Ok,NaN,NaN,2025-01-01,NaN,12.0,...,NaN,535.00,64.20,NaN,309.80,909.00,51.922260,52.895120,5.115940,5.646490
8,552107,35.42,59.0,135.0,Ok,NaN,NaN,2025-01-01,NaN,12.0,...,NaN,535.00,64.20,NaN,149.80,749.00,51.922260,52.895120,5.115940,5.646490
9,552111,NaN,88.0,74.0,Ok,Ja,NaN,2025-01-01,NaN,12.0,...,NaN,970.00,116.40,27.72,662.60,1776.72,50.913860,52.895120,2.909990,5.646490


CIP Laden & CIP Lossen

In [717]:
# data["CIP Laden"] = data["CIP Laden"].fillna("Nee")
# data["CIP Lossen"] = data["CIP Lossen"].fillna("Nee")
# data[["CIP Laden", "CIP Lossen"]]

Afleverstatus

In [718]:
# print(data["Afleverstatus"].unique())
# data = pd.get_dummies(data, columns=["Afleverstatus"])

#data.drop(["Afleverstatus"], axis=1, inplace=True)

Calculatie Gewicht

In [719]:
data["Calculatie gewicht"] = data["Calculatie gewicht"] * 1000
data["Calculatie gewicht"].fillna(data["Laadgewicht"], inplace=True)

ADR-zending

In [720]:
data["ADR-zending"].fillna("Nee", inplace=True)

Kosten

In [721]:
data["Kosten"].fillna(0, inplace=True)

Brandstoftoeslag percentage

In [722]:
data["Brandstoftoeslag percentage"].fillna(0, inplace=True)

afwijkingen

In [723]:
data["# afwijkingen"].fillna(0, inplace=True)
data.rename(columns={"# afwijkingen": "afwijking_vracht"}, inplace=True)

CMR Ontvangen & CMR bij factuur

In [724]:
# data["CMR ontvangen"].fillna("Nee", inplace=True)
# data["CMR bij factuur"].fillna("Nee", inplace=True)

Losland & laadland

In [725]:
data["Losland"].unique()
missing_count = data["Losland"].isna().sum()
print("Missing: ", missing_count)
data["Losland"].fillna("Onbekend", inplace=True)
data = pd.get_dummies(data, columns=["Losland"])


Missing:  295


In [726]:
data["Laadland"].unique()
missing_count = data["Laadland"].isna().sum()
print("Laadland: ", missing_count)
data["Laadland"].fillna("Onbekend", inplace=True)
data = pd.get_dummies(data, columns=["Laadland"])


Laadland:  117


---

# **6.0 - Feature Engineering**

---

Afwijkingen in Gewicht

In [727]:
data["afwijking_gewicht"] = data["Laadgewicht"] - data["Losgewicht"]


data["afwijking_gewicht"] = np.where(
    (data["afwijking_gewicht"].abs() >= data["Laadgewicht"] * 0.95) |
    (data["afwijking_gewicht"].abs() >= data["Losgewicht"] * 0.95),
    0, data["afwijking_gewicht"])

print(data["afwijking_gewicht"].describe())
data["afwijking_gewicht"].head(200)

data.drop(columns=["Laadgewicht", "Losgewicht"])


count    35428.000000
mean        58.962741
std        911.024297
min     -13720.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      17100.000000
Name: afwijking_gewicht, dtype: float64


,Dossiernr,Calculatie gewicht,Duur laden,Duur lossen,Routeberek. resultaat,ADR-zending,Productomschrijving,Financiële datum,Artikel merken/nrs.,Brandstoftoeslag percentage,...,Laadland_AT,Laadland_BE,Laadland_DE,Laadland_ES,Laadland_FR,Laadland_IT,Laadland_NL,Laadland_Onbekend,Laadland_SE,afwijking_gewicht
0,551801,33100.0,30.0,214.0,Ok,Ja,Wei proteine concentraat,2025-01-01,Wei proteine concentraat,12.0,...,False,False,False,False,False,False,True,False,False,0.0
1,552062,33100.0,53.0,134.0,Ok,Nee,NaN,2025-01-01,NaN,12.0,...,False,False,False,False,False,False,True,False,False,0.0
2,552063,33140.0,64.0,146.0,Ok,Nee,NaN,2025-01-01,NaN,11.0,...,False,False,False,False,False,False,True,False,False,0.0
3,552076,25520.0,107.0,62.0,Ok,Ja,WPC 65,2025-01-01,WPC 65,10.0,...,False,False,False,False,False,False,True,False,False,60.0
4,552090,34860.0,177.0,70.0,Ok,Nee,Magere melk,2025-01-01,Magere melk,11.0,...,False,False,False,False,False,False,True,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38546,593957,NaN,NaN,NaN,Ok,Ja,Wei proteine concentraat,2025-08-31,NaN,11.0,...,False,False,False,False,False,False,True,False,False,NaN
38547,594024,35000.0,NaN,NaN,Ok,Nee,Magere melk,2025-08-31,NaN,11.0,...,False,False,False,False,False,False,True,False,False,NaN
38548,594025,35000.0,NaN,NaN,Ok,Nee,Magere melk,2025-08-31,NaN,11.0,...,False,False,False,False,False,False,True,False,False,NaN
38549,594322,NaN,NaN,NaN,Ok,Nee,NaN,2025-08-31,NaN,0.0,...,False,False,False,False,False,False,True,False,False,NaN


Route Haversine

In [728]:
def calculate_haversine(row):
    start = (row['Breedtegraad laadadres'], row['Lengtegraad laadadres'])
    end = (row['Breedtegraad losadres'], row['Lengtegraad losadres'])
    return haversine(start, end, unit=Unit.KILOMETERS)

data['afstand_haversine'] = data.apply(calculate_haversine, axis=1)
data.drop(labels=['Breedtegraad laadadres', 'Lengtegraad laadadres', 'Breedtegraad losadres', 'Lengtegraad losadres'], axis=1, inplace=True)

In [ ]:
data["afwijking_afstand"] = data["Afstand"].fillna(0) - data["afstand_haversine"]
data["afwijking_afstand"].describe()

KeyError: 'Afwijking'

Binary Encoding

In [730]:
def convert_yes_no_to_binary(df):
    yes_no_map = {'yes': True, 'no': False, 'ja': True, 'nee': False}
    for col in df.columns:
        if df[col].dtype == object:
            unique_vals = df[col].dropna().str.lower().unique()
            if set(unique_vals).issubset(yes_no_map.keys()):
                df[col] = df[col].str.lower().map(yes_no_map)
    return df

data = convert_yes_no_to_binary(data)

Datatypes

In [731]:

def optimize_numeric_floats(df):
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    for col in df.select_dtypes(include=[np.number]).columns:
        col_data = df[col]
        if np.issubdtype(col_data.dtype, np.integer):
            df[col] = pd.to_numeric(col_data, downcast='integer')
        elif np.issubdtype(col_data.dtype, np.floating):
            if col_data.min() >= np.finfo(np.float16).min and col_data.max() <= np.finfo(np.float16).max:
                df[col] = col_data.astype(np.float16)
            else:
                df[col] = col_data.astype(np.float32)
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Memory usage reduced from {start_mem:.2f} MB to {end_mem:.2f} MB")
    print(df.dtypes)
    return df

data = optimize_numeric_floats(data)

Memory usage reduced from 59.30 MB to 54.52 MB
Dossiernr                  int32
Calculatie gewicht       float32
Duur laden               float16
Duur lossen              float16
Routeberek. resultaat     object
                          ...   
Laadland_Onbekend           bool
Laadland_SE                 bool
afwijking_gewicht        float16
afstand_haversine        float16
afwijking_afstand        float16
Length: 70, dtype: object


---

# **7.0 - Clean Dataset Summary**

---

In [732]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38551 entries, 0 to 38550
Data columns (total 70 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Dossiernr                     38551 non-null  int32         
 1   Calculatie gewicht            38170 non-null  float32       
 2   Duur laden                    34527 non-null  float16       
 3   Duur lossen                   34827 non-null  float16       
 4   Routeberek. resultaat         38551 non-null  object        
 5   ADR-zending                   38551 non-null  bool          
 6   Productomschrijving           16870 non-null  object        
 7   Financiële datum              38551 non-null  datetime64[ns]
 8   Artikel merken/nrs.           15541 non-null  object        
 9   Brandstoftoeslag percentage   38551 non-null  float16       
 10  Gepl. losdatum/tijd           36036 non-null  datetime64[ns]
 11  Gepl. laaddatum/tijd        

In [733]:
data.describe()

,Dossiernr,Calculatie gewicht,Duur laden,Duur lossen,Financiële datum,Brandstoftoeslag percentage,Gepl. losdatum/tijd,Gepl. laaddatum/tijd,Aangemaakt op,Kosten,...,Vaste prijs,Eenheidsprijs,Bedrag excl.,Brandstoftoeslag bedrag,Brandstoftoesl. extra kosten,Extra kosten bedrag,Totaalbedrag,afwijking_gewicht,afstand_haversine,afwijking_afstand
count,38551.000000,3.817000e+04,34527.0,34827.0,38551,3.855100e+04,36036,36037,38551,38551.00,...,20080.0,1.629100e+04,36343.0,28134.00000,3588.000000,19090.00,36823.00,35428.0,38233.0000,3.823300e+04
mean,573409.614770,2.776058e+04,inf,inf,2025-05-02 01:13:59.791444992,inf,2025-05-01 15:22:07.250582784,2025-05-01 03:33:26.911896064,2025-04-27 23:52:41.081165312,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
min,174567.000000,-3.548000e+04,0.0,0.0,2025-01-01 00:00:00,-3.429688e+00,2010-01-02 03:34:05,2010-01-01 08:14:29,2024-10-08 14:22:49,0.00,...,-2142.0,-8.525000e+01,-2156.0,-345.00000,0.000000,-443.75,-2840.00,-13720.0,0.0000,-3.700000e+01
25%,562918.000000,2.442000e+04,51.0,35.0,2025-03-05 00:00:00,3.810547e+00,2025-03-05 11:21:33.500000,2025-03-05 06:43:03,2025-02-28 12:02:00,172.75,...,341.0,9.851562e+00,330.5,33.46875,14.882812,20.00,388.75,0.0,64.2500,2.357812e+01
50%,573481.000000,2.774000e+04,68.0,54.0,2025-05-02 00:00:00,1.100000e+01,2025-05-01 07:38:46,2025-04-30 15:24:00,2025-04-28 10:48:42,425.50,...,575.0,1.345312e+01,495.0,57.28125,20.250000,87.50,577.50,0.0,98.1875,3.881250e+01
75%,583879.500000,3.396000e+04,93.0,88.0,2025-06-30 00:00:00,1.379688e+01,2025-06-30 08:52:55,2025-06-29 13:39:43,2025-06-26 10:56:12.500000,732.50,...,775.0,2.040625e+01,713.0,81.75000,25.906250,176.25,859.50,0.0,171.6250,5.646875e+01
max,594713.000000,3.500000e+07,17264.0,7200.0,2025-08-31 00:00:00,2.000000e+01,2025-09-01 01:30:00,2025-09-01 00:00:00,2025-08-27 12:31:50,6500.00,...,6744.0,8.950000e+02,6744.0,742.00000,225.250000,62656.00,63296.00,17104.0,1343.0000,4.280000e+02
std,12522.573516,1.791793e+05,inf,inf,NaN,6.308594e+00,NaN,NaN,NaN,inf,...,inf,inf,inf,inf,10.953125,inf,inf,inf,inf,inf


In [734]:
data.memory_usage()

Index                    132
Dossiernr             154204
Calculatie gewicht    154204
Duur laden             77102
Duur lossen            77102
                       ...  
Laadland_Onbekend      38551
Laadland_SE            38551
afwijking_gewicht      77102
afstand_haversine      77102
afwijking_afstand      77102
Length: 71, dtype: int64

In [735]:
feature_summary(data)

,Feature,Num_Unique,Num_Missing
0,Interne opmerkingen,26,38479
1,Brandstoftoesl. extra kosten,336,34963
2,Artikel merken/nrs.,738,23010
3,Eenheidsprijs,765,22260
4,Productomschrijving,753,21681
...,...,...,...
65,Losland_BE,2,0
66,Losland_CH,2,0
67,Losland_CZ,2,0
68,Losland_DE,2,0


In [736]:
def compare_dataframes(df_raw, df_cleaned):
    dropped = set(df_raw.columns) - set(df_cleaned.columns)
    added = set(df_cleaned.columns) - set(df_raw.columns)
    print("Dropped columns:", dropped)
    print("Added columns:", added)
    common_cols = df_raw.columns.intersection(df_cleaned.columns)
    diff_summary = []
    for col in common_cols:
        raw_dtype = df_raw[col].dtype
        clean_dtype = df_cleaned[col].dtype
        missing_raw = df_raw[col].isna().sum()
        missing_clean = df_cleaned[col].isna().sum()
        if np.issubdtype(df_raw[col].dtype, np.number) and np.issubdtype(df_cleaned[col].dtype, np.number):
            diff = (df_raw[col] - df_cleaned[col]).abs()
            max_diff = diff.max()
            mean_diff = diff.mean()
        else:
            max_diff = mean_diff = np.nan
        diff_summary.append({
            "column": col,
            "dtype_raw": raw_dtype,
            "dtype_cleaned": clean_dtype,
            "missing_raw": missing_raw,
            "missing_clean": missing_clean,
            "max_diff": max_diff,
            "mean_diff": mean_diff
        })
    summary_df = pd.DataFrame(diff_summary)
    print("\nComparison Summary:")
    print(summary_df)
    return summary_df
compare_dataframes(dataset, data)

Dropped columns: {'Einddatum lossen', 'Geen autotarief', 'Gepl. laadtijd', 'Crediteur lossen', 'Eindtijd laden', 'Afleverstatus opm.', 'Ritnr lossen', 'Factuurmethode', 'Status', 'Startdatum laden', 'CMR nr.', 'Laadadres2', 'Trailer lossen', 'Factuurnr', 'Lengtegraad laadadres', 'Factuuropmerkingen', 'Coördinaten laadadres handmatig', 'CIP Lossen', 'Temperatuur', '# afwijkingen', 'Legnr', 'CIP Laden', 'TariffUnitAmountRO', 'Volgnr', 'Afwijkend tariefaantal', 'Eiwit', 'Starttijd lossen', 'Oorspr. zendingnr', '# Beschadigd', 'Breedtegraad losadres', 'EDI referentie', 'Temp. t/m', 'Starttijd laden', 'Losadres2', 'Nippelklant', 'Geochecksum laadadres', 'Gecontroleerd', 'Debiteur', 'Lospostcode', 'Chauffeur lossen', 'Factuurcode', '% Droge stof', 'Laadtijd t/m', 'Ordersoort', 'Neutraal laadadres', 'Klaar voor factuur', 'Kritische nippelklant', 'Brandstoftoeslag handmatig', '# Geweigerd', 'Chauffeur losinfo', 'Breedtegraad laadadres', 'Startdatum lossen', 'Laadreferentie', 'CMR ontvangen', '

,column,dtype_raw,dtype_cleaned,missing_raw,missing_clean,max_diff,mean_diff
0,Dossiernr,int64,int32,0,0,0.000000e+00,0.000000e+00
1,Totaalbedrag,float64,float16,1728,1728,6.480000e+00,1.212803e-01
2,Extra kosten bedrag,float64,float16,19461,19461,1.119000e+01,1.743388e-02
3,Brandstoftoesl. extra kosten,float64,float16,34963,34963,4.500000e-02,3.116225e-03
4,Brandstoftoeslag bedrag,float64,float16,10417,10417,1.200000e-01,1.108206e-02
5,Bedrag excl.,float64,float16,2208,2208,2.000000e+00,8.517321e-02
6,Eenheidsprijs,float64,float16,22260,22260,3.500000e-02,2.477010e-03
7,Vaste prijs,float64,float16,18471,18471,2.000000e+00,8.091494e-02
8,Laadgewicht,float64,float16,2901,2901,1.600000e+01,5.075231e+00
9,Losgewicht,float64,float16,3060,3060,1.600000e+01,4.967259e+00
